## Preferred to be run on **[colab.research.google.com](https://colab.research.google.com)**

# row: away , column : home

### Randomize or predefine the `distance_matrix`

In [2]:
import random
n = int(input())
distance_matrix = [[random.randint(0,20) for i in range(n)] for i in range(n)]
for i in range(n):
    distance_matrix[i][i] = 0
    for j in range(n):
        distance_matrix[j][i] = distance_matrix[i][j]

# distance_matrix = [[0, 12, 8, 5, 5, 9], [12, 0, 9, 19, 4, 10], [8, 9, 0, 10, 15, 1], [5, 19, 10, 0, 18, 5], [5, 4, 15, 18, 0, 8], [9, 10, 1, 5, 8, 0]]
distance_matrix = [[0, 2, 3, 10], [2, 0, 3, 6], [3, 3, 0, 9], [10, 6, 9, 0]]

print(distance_matrix)

4
[[0, 2, 3, 10], [2, 0, 3, 6], [3, 3, 0, 9], [10, 6, 9, 0]]


# Backtrack algorithm


In [5]:
def find_min(matrix:list):
    ans = float('inf')
    for row in matrix:
        for entry in row:
            if entry != 0 and entry < ans:
                ans = entry
    return ans

In [3]:
import random
import copy

class Node():
    def __init__(self,matrix:list,dis,week,tracking_pos) -> None:
        self.matrix = matrix
        self.dis = dis
        self.week = week
        self.tracking_pos = tracking_pos
                        

def get_sub_node(distance_matrix,node:Node):
        """ returns the list contains all tuples: (matrix,pos,dis) """
        ans_matrix = []
        final_ans = []

        queue = [(node,0)]

        current_week = node.week + 1
        N = len(distance_matrix)
        assert N%2 == 0, "The number of teams must be even"
        
        while queue:
            temp,match = queue.pop()
            matrix = temp.matrix
            pos = temp.tracking_pos
            dis = temp.dis
            if match == N//2:
                if temp.matrix not in ans_matrix:
                    ans_matrix.append(temp.matrix)
                    final_ans.append((temp.matrix,temp.tracking_pos,temp.dis))
            else:
                for i in range(N):
                    if current_week not in matrix[i] and current_week not in [matrix[j][i] for\
                        j in range(N)]:
                        for j in range(N):
                            if i!=j and current_week not in matrix[j] and current_week not \
                                in [matrix[i][j] for i in range(N)]:
                                if matrix[i][j] == None:
                                    sub_matrix = copy.deepcopy(matrix)
                                    sub_matrix[i][j] = current_week
                                    sub_pos = copy.deepcopy(pos)
                                    sub_pos[i],sub_pos[j] = j,j
                                    sub_dis = dis + distance_matrix[pos[i]][j] + distance_matrix[pos[j]][j]
                                    sub_Node = Node(sub_matrix,sub_dis,current_week,sub_pos)
                                    queue.append((sub_Node,match+1))
        return final_ans

def backtrack_solve(input:list):
    N = len(input)
    queue = []
    ## tracking the best value in the tree
    best_value = float('inf')
    
    best_config = None
    ##
    ##initialize the root of the tree.
    root_matrix = [[None for i in range(N)] for i in range(N)]
    for i in range(N):
        root_matrix[i][i] = 0
    root_dis = 0
    week = 0
    root_tracking_pos = [i for i in range(N)]
    root = Node(root_matrix,root_dis,week,root_tracking_pos)
    queue.append(root)
    while queue:
        temp = queue.pop()
        assert isinstance(temp, Node)

        current_week = temp.week
        if current_week == 2*N - 2:
            if temp.dis < best_value:
                best_config = temp.matrix
                best_value = temp.dis
                print(f"Found better configuration, the new best value is : {best_value}")
                print(f"The new best config is {best_config}")
                print()

        else:
            next_week = current_week + 1
            list_of_nodes = get_sub_node(input,temp)
            for item in list_of_nodes:
                sub_matrix,sub_pos,sub_dis = item
                if sub_dis < best_value:
                    sub_Node = Node(sub_matrix,sub_dis,next_week,sub_pos)
                    queue.append(sub_Node)

    return best_config, best_value

# distance_matrix = [[random.randint(0,10) for i in range(4)] for i in range(4)]
# for i in range(4):
#     distance_matrix[i][i] = 0
#     for j in range(4):
#         distance_matrix[j][i] = distance_matrix[i][j]

print(distance_matrix)
print()
backtrack_solve(input=distance_matrix)


[[0, 2, 3, 10], [2, 0, 3, 6], [3, 3, 0, 9], [10, 6, 9, 0]]

Found better configuration, the new best value is : 79
The new best config is [[0, 3, 2, 1], [6, 0, 1, 2], [5, 4, 0, 3], [4, 5, 6, 0]]

Found better configuration, the new best value is : 75
The new best config is [[0, 3, 1, 2], [6, 0, 2, 1], [5, 4, 0, 3], [4, 5, 6, 0]]

Found better configuration, the new best value is : 71
The new best config is [[0, 3, 1, 2], [6, 0, 5, 1], [4, 2, 0, 3], [5, 4, 6, 0]]



([[0, 3, 1, 2], [6, 0, 5, 1], [4, 2, 0, 3], [5, 4, 6, 0]], 71)

# BeamSearch algorithm

In [6]:
def beam_solve(input,k):
    N = len(input)
    queue = []
    ## tracking the best value in the tree
    c_min = find_min(input)

    ##
    ##initialize the root of the tree.
    root_matrix = [[None for i in range(N)] for i in range(N)]
    for i in range(N):
        root_matrix[i][i] = 0
    root_dis = 0
    week = 0
    root_tracking_pos = [i for i in range(N)]
    root = Node(root_matrix,root_dis,week,root_tracking_pos)
    queue.append(root)
    best = None
    
    while True:
        nodes = copy.deepcopy(queue) 
        queue = []
        if nodes[0].week == 2*N - 2:
            for node in nodes:
                if best == None:
                    best = node.matrix,node.dis
                else:
                    if node.dis < best[1]:
                        best = node.matrix,node.dis
            break
        else:
            next_week = nodes[0].week + 1
            for node in nodes:
                lis = get_sub_node(input,node)
                sub_nodes = [Node(sub_matrix,sub_dis,next_week,sub_pos) for \
                    (sub_matrix,sub_pos,sub_dis) in lis]
                queue += sub_nodes
            queue = sorted(queue,key=lambda x: x.dis)[:k]

    return best

# distance_matrix = [[random.randint(0,10) for i in range(8)] for i in range(8)]
# for i in range(8):
#     distance_matrix[i][i] = 0
#     for j in range(8):
#         distance_matrix[j][i] = distance_matrix[i][j]

config,val = beam_solve(distance_matrix, 10) 
print(f'The reasonable cost when using Beam Search is {val}')

The reasonable cost when using Beam Search is 79


# Local Search

In [7]:
import traceback
import random
import numpy 

def generate(N):

    mat_0 = list()
    for i in range(N):
        row = [None]*N
        row[i] = 0
        mat_0.append(row)

    all_coors_0 = list((a,b) for a in range(0,N) for b in range(0,N))
    for i in range(N):
        all_coors_0.remove((i,i))

    n_choices_0 = dict()
    choices_0 = dict()
    for i in range(1,N+1):
        n_choices_0[i] = N-1
        choices_0[i] = [a for a in range(1,N+1) if a != i]
    
    univ_tab = dict()

    univ_tab[0] = [copy.deepcopy(n_choices_0), copy.deepcopy(choices_0), copy.deepcopy(all_coors_0), copy.deepcopy(mat_0)]

    global result
    result = []
    
    global S 
    S = False

    def construct_mat(n):
        global result
        global S
        if n == 2*N - 2:
            S = True
            result = univ_tab[n][3]
        else:
            nextweek = construct_week(n+1)
            if nextweek == False:
                construct_mat(n)
            elif nextweek == 'Reset':
                construct_mat(n-1)
            else:
                univ_tab[n+1] = copy.deepcopy(nextweek)
                construct_mat(n+1)

    # functions for contructing week 

    def create_priority_queue(n_choices_f):
        L = list()
        for a,b in n_choices_f.items():
            L.append((b,a))

        L.sort()

        Q = list()
        for item in L:
            Q.append(item[1])
        return Q

    def check_choices(choices_copy):
        L = list()
        for key in choices_copy.keys():
            if len(choices_copy[key]) == 0:
                L.append(key)
        for key in L:
            choices_copy.pop(key)

    def create_n_choices(choices):
        n_choices_f = dict()
        for key in choices.keys():
            n_choices_f[key] = len(choices[key])
        
        return n_choices_f
        
    def choose_match(a: int, n_choices: dict, choices: dict, choices_copy: dict, all_coors: list, rest: list, queue: list, mat: list, week: int, p_queue: list, b = None):
        
        if b == None:
            b = random.choice(choices_copy[a])

        rest.remove(a)
        rest.remove(b)
        queue.remove(a)
        queue.remove(b)
        p_queue = copy.deepcopy(queue)

        for key in choices_copy.keys():
            if a in choices_copy[key]:
                choices_copy[key].remove(a)
            if b in choices_copy[key]:
                choices_copy[key].remove(b)
        choices_copy.pop(a)
        choices_copy.pop(b)

        RC = []
        if (a-1,b-1) in all_coors:
            RC.append((a,b))
        if (b-1,a-1) in all_coors:
            RC.append((b,a))
        
        (c,d) = random.choice(RC)

        all_coors.remove((c-1,d-1))

        if len(RC) == 1:
            n_choices[a] -= 1
            n_choices[b] -= 1
            choices[a].remove(b)
            choices[b].remove(a)

        mat[c-1][d-1] = week
        
    # constructing week

    def construct_week(m):

        univ = copy.deepcopy(univ_tab[m-1])
        # n_choices = univ[0]
        # choices = univ[1]
        # all_coors = univ[2]
        # mat = univ[3]
        
        choices_copy = copy.deepcopy(univ[1])

        queue = create_priority_queue(univ[0])

        p_queue = queue[:]
        rest = [i for i in range(1,N+1)]

        while len(p_queue) >=1:

            if len(rest) <= 2:
                if len(rest) == 1:
                    return False
                else:
                    if rest[1] not in univ[1][rest[0]]:
                        return False
                    else:
                        check_choices(choices_copy)
                        choose_match(rest[1], univ[0], univ[1], choices_copy, univ[2], rest, queue, univ[3], m, p_queue, rest[0])
                        return univ
            else:
                check_choices(choices_copy)
                queue = create_priority_queue(create_n_choices(choices_copy))
                p_queue = queue[:]

                i = p_queue[0]

                choose_match(i, univ[0], univ[1], choices_copy, univ[2], rest, queue, univ[3], m, p_queue)

                if len(choices_copy.keys()) % 2 != 0:
                    return 'Reset'

    def do_until_true():
        global S
        while S == False:
            try:
                construct_mat(0)
            except Exception:
                do_until_true()
    
    do_until_true()

    return result


def get_neighbors(initialize, d_mat): #initialize: mat,cost
    '''return list of (mat,cost)'''
    mat, cost = initialize

    M = copy.deepcopy(mat)
    N = len(M)

    all_neighbors = list()

    def has_a_common(c_1, c_2):
        state = False
        for i in c_1:
            for j in c_2:
                if i == j:
                    state = True
        return state

    def swap(coor_1: tuple, coor_2: tuple, M: list):

        inp_m = copy.deepcopy(M)

        N = len(inp_m)
        
        def common_team(c_1, c_2):
            for i in c_1:
                for j in c_2:
                    if i == j:
                        return i

        def value_swap(c_1, c_2, m):

            if c_1 == None or c_2 == None:
                print(m)
                print(inp_m)
                print(mat)
            temp = m[c_1[0]][c_1[1]]
            m[c_1[0]][c_1[1]] = m[c_2[0]][c_2[1]]
            m[c_2[0]][c_2[1]] = temp

        def other_team(c, t):
            for i in c:
                if i != t:
                    return i

        def check(t, m):
            indexes = list((t, i) for i in range(N) if i!= t) + list((i, t) for i in range(N) if i!= t)
            value = list()
            for i in indexes:
                if inp_m[i[0]][i[1]] not in value:
                    value.append(m[i[0]][i[1]])
            if len(value) == 2*N - 2:
                return True
            else:
                return False

        def find_index(c,t,m):
            match = m[c[0]][c[1]]
            indexes = list((t, i) for i in range(N) if i!= t) + list((i, t) for i in range(N) if i!= t)
            indexes.remove(c)
            for i in indexes:
                if m[i[0]][i[1]] == match:
                    return i

        if has_a_common(coor_1, coor_2):

            match_1 = inp_m[coor_1[0]][coor_1[1]]
            match_2 = inp_m[coor_2[0]][coor_2[1]]

            team_1 = common_team(coor_1, coor_2)
            team_2 = team_1

            value_swap(coor_1, coor_2, inp_m)

            state_1 = check(other_team(coor_1, team_1), inp_m)
            state_2 = check(other_team(coor_2, team_2), inp_m)

            while (state_1 and state_2) == False:
                match_1, match_2 = match_2, match_1

                team_1 = other_team(coor_1, team_1)
                team_2 = other_team(coor_2, team_2)

                coor_1 = find_index(coor_1, team_1, inp_m)
                coor_2 = find_index(coor_2, team_2, inp_m)

                value_swap(coor_1, coor_2, inp_m)

                state_1 = check(other_team(coor_1, team_1), inp_m)
                state_2 = check(other_team(coor_2, team_2), inp_m)

            return inp_m, compute_cost(d_mat,inp_m)
        
        else:
            match_1 = inp_m[coor_1[0]][coor_1[1]]
            match_2 = inp_m[coor_2[0]][coor_2[1]]

            for i in inp_m:
                for j in i:
                    if j == match_1:
                        j = match_2
                    elif j == match_2:
                        j = match_1

            return inp_m, compute_cost(d_mat, inp_m)

    def team_swap(teams):
        a,b = teams
        a -= 1
        b -= 1
        Y = copy.deepcopy(M)
        Y_1 = numpy.array(copy.deepcopy(Y))
        
        def replace_in_mat(x,y,S):
            temp = S[x].copy()
            S[x] = S[y].copy()
            S[y] = temp.copy()

            S[x][y] = S[x][x]
            S[x][x] = 0

            S[y][x] = S[y][y]
            S[y][y] = 0

        replace_in_mat(a,b,Y_1)
        Y_1 = Y_1.T.copy()
        
        replace_in_mat(a,b,Y_1)
        Y_1 = Y_1.T.copy()

        Y = copy.deepcopy(Y_1.tolist())

        return Y, compute_cost(d_mat, Y)

    iterate = list((i,j) for i in range(1,N + 1) for j in range(i, N + 1))

    for coor in iterate:
        mat_1, cost_1 = copy.deepcopy(team_swap(coor))
        all_neighbors.append((mat_1, cost_1))

    iterate = []
    all_coors = [(j,i) for i in range(0,N) for j in range(0,N) if i!= j]
    while len(all_coors) > 0:
        coor = all_coors[0]
        all_coors.pop(0)
        sub_iterate = [(coor, i) for i in all_coors]
        iterate += sub_iterate
    
    for coors in iterate:
        coor_1 = coors[0]
        coor_2 = coors[1]
        mat_1, cost_1 = copy.deepcopy(swap(coor_1, coor_2, M))
        all_neighbors.append((mat_1, cost_1))

    return all_neighbors


def compute_cost(distance_matrix,matrix):
    '''compute cost of a solution'''
    num_of_teams = len(matrix)
    cost = 0
    for team in range(num_of_teams):
        current_pos = team
        team_dis = 0
        current_week = 0
        while current_week < 2*num_of_teams - 2:
            current_week = current_week + 1
            flag = True
            for index in range(num_of_teams):
                if matrix[team][index] == current_week:
                    old_pos = current_pos
                    current_pos = index
                    team_dis += distance_matrix[old_pos][current_pos]
                    flag = False
                    break
            if flag: # da san nha
                old_pos = current_pos
                current_pos = team
                team_dis += distance_matrix[old_pos][current_pos]

        cost += team_dis
        
    return cost
    


In [8]:
def local_solve(input):
    # random inial
    #input: distance matrix
    #return the schedule: matrix , cost
    N = len(input)
    random_matrix = generate(N)
    cost = compute_cost(input,random_matrix)
    initialize = random_matrix,cost
    flag = True
    while flag:
        neighbors = get_neighbors(initialize, input)
        for neighbor in neighbors:
            #neighbor: matrix:schedule,cost
            schedule,cost = neighbor
            flag = False
            if cost < initialize[1]:
                initialize = neighbor
                flag = True
    return initialize[0],initialize[1]
config , val = local_solve(distance_matrix)
print(f'The reasonable cost when using Local Search is {val}')


The reasonable cost when using Local Search is 79


# Meta-heuristic algorithm

### iterative local search

In [9]:
def iterative_local_solve(input,K):
    best_config = None
    best_cost = float('inf')
    for i in range(K):
        matrix,cost = local_solve(input)
        if cost < best_cost:
            best_cost = cost
            best_config = matrix

    return best_config,best_cost
mat,cost = iterative_local_solve(distance_matrix,K=10)
print(f'The reasonable cost when using Iterative Local Search is {cost}')


The reasonable cost when using Iterative Local Search is 73


### Gibb sampling: but this method is not efficient yet

In [10]:
def gibb_sampling(input,K,factor):

    #initialize start position
    mat = generate(len(input))
    current_position = (mat,compute_cost(input,mat))
    #
    best_mat = None
    best_cost = float('inf')
    for i in range(K):
        #choosing a random number:
        num = random.random()

        list_of_neighs = get_neighbors(current_position, input) #mat,cost
        sum_of_cost = sum(1/(factor**(x[1])) for x in list_of_neighs)
        accumulate = 0
        for i in range(len(list_of_neighs)):
            mat,cost = list_of_neighs[i]
            accumulate += 1/(factor ** cost)
            port = accumulate / sum_of_cost
            if port > num:
                current_position = list_of_neighs[i-1]
                if current_position[1] < best_cost:
                  best_cost = current_position[1]
                  best_mat = current_position[0]

                break
    return best_mat,best_cost


In [11]:
mat,cost = gibb_sampling(distance_matrix,1000,1000)
print(f'The reasonable cost when using Gibb_sampling is {cost}')

The reasonable cost when using Gibb_sampling is 83


# OR-Tools

In [ ]:
%pip install ortools

In [14]:
from ortools.sat.python import cp_model
import copy
import time

class VarArrayAndObjectiveSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions (objective, variable values, time)."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.__start_time = time.time()

    def on_solution_callback(self):
        """Called on each new solution."""
        current_time = time.time()
        obj = self.ObjectiveValue()
        print('Solution %i, time = %0.2f s, objective = %i' %
              (self.__solution_count, current_time - self.__start_time, obj))
        for i in range(len(self.__variables)):
            for j in range(len(self.__variables)):
                print(self.Value(self.__variables[i][j]), end = ' ')
            print()
        print()
        self.__solution_count += 1

    def solution_count(self):
        """Returns the number of solutions found."""
        return self.__solution_count

def OR_Tools_solve(input, time_limit = 300):
    
    distance_matrix = input
    N = len(input)

    model = cp_model.CpModel()

    x = []
    for i in range(N):
        row = [None]*N
        row[i] = 0
        x.append(row)

    for i in range(N):
        for j in range(N):
            if i == j:
                x[i][i] = model.NewIntVar(0,0,'x[%i][%i]' % (i,i))
            else:
                x[i][j] = model.NewIntVar(0,2*N - 2, 'x[%i][%i]' %(i,j))

    for i in range(N):
        c = list(x[j][i] for j in range(N))
        c.pop(i)
        cnr = x[i] + c
        model.AddAllDifferent(cnr)
    
    y = []
    for i in range(N):
        y_row = []
        for j in range(2*N - 1):
            row = [None]*(2*N - 1)
            row[i] = 0
            y_row.append(row)
        y.append(y_row)
    for i in range(N):
        for j in range(2*N - 1):
            for k in range(2*N - 1):
                y[i][j][k] = model.NewBoolVar('y[%i][%i][%i]' % (i,j,k))
    

    obj_func = 0

    for team in range(N):

        indexes = list((team,j) for j in range(0,N) if team!= j) + list((j,team) for j in range(0,N) if team!= j)
        index_0 = (team,team)

        for i in range(len(indexes)):
            model.Add(x[indexes[i][0]][indexes[i][1]] == x[index_0[0]][index_0[1]] + 1).OnlyEnforceIf(y[team][0][i + 1])
            model.Add(x[indexes[i][0]][indexes[i][1]] != x[index_0[0]][index_0[1]] + 1).OnlyEnforceIf(y[team][0][i + 1].Not())
        for i in range(len(indexes)):
            indexes_copy = copy.deepcopy(indexes)
            for j in range(len(indexes_copy)):
                if i != j:
                    model.Add(x[indexes[i][0]][indexes[i][1]] == x[indexes_copy[j][0]][indexes_copy[j][1]] - 1).OnlyEnforceIf(y[team][i + 1][j + 1])
                    model.Add(x[indexes[i][0]][indexes[i][1]] != x[indexes_copy[j][0]][indexes_copy[j][1]] - 1).OnlyEnforceIf(y[team][i + 1][j + 1].Not())
        
        others = list(i for i in range(N) if i!= team)
        for i in range(2*N - 1):
            for j in range(2*N - 1):
                if i == 0:
                    if j == 0:
                        obj_func += 0
                    elif j <= N-1:
                        obj_func += distance_matrix[team][others[j - 1]] * y[team][0][j]
                    else:
                        obj_func += distance_matrix[team][team] * y[team][0][j]
                elif i <= N-1:
                    if j == 0:
                        obj_func += 0
                    elif j <= N-1:
                        obj_func += distance_matrix[others[i - 1]][others[j - 1]] * y[team][i][j]
                    else:
                        obj_func += distance_matrix[others[i - 1]][team] * y[team][i][j]
                else:
                    if j == 0:
                        obj_func += 0
                    elif j <= N-1:
                        obj_func += distance_matrix[team][others[j - 1]] * y[team][i][j]
                    else:
                        obj_func += distance_matrix[team][team] * y[team][i][j]

    model.Minimize(obj_func)
    
    solver = cp_model.CpSolver()
    solution_printer = VarArrayAndObjectiveSolutionPrinter(x)
    solver.parameters.max_time_in_seconds = time_limit
    status = solver.Solve(model, solution_printer)
    
    result = list()

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f'Minimum of objective function: {solver.ObjectiveValue()}\n')
        for i in range(N):
            for j in range(N):
                print(solver.Value(x[i][j]), end = ' ')
            print()
        
        for i in range(N):
          sub_result = list()
          for j in range(N):
            sub_result.append(solver.Value(x[i][j]))
          result.append(sub_result)

    else:
        print('No solution found.')
    
    print('\nStatistic')
    print(f'  status : {solver.StatusName(status)}')
    print(f'  conflicts: {solver.NumConflicts()}')
    print(f'  branches : {solver.NumBranches()}')
    print(f'  wall time: {solver.WallTime()} s')
    print(f'  Reasonable cost using OR-Tools: {solver.ObjectiveValue()}')
    print()

    return result, solver.ObjectiveValue()

OR_Tools_solve(distance_matrix, 300)

Solution 0, time = 0.06 s, objective = 81
0 5 6 2 
3 0 4 1 
1 2 0 3 
4 6 5 0 

Solution 1, time = 0.07 s, objective = 77
0 3 6 2 
5 0 4 1 
1 2 0 3 
4 6 5 0 

Solution 2, time = 0.07 s, objective = 75
0 3 1 2 
6 0 2 1 
5 4 0 3 
4 5 6 0 

Solution 3, time = 0.21 s, objective = 71
0 3 1 2 
6 0 5 1 
4 2 0 3 
5 4 6 0 

Minimum of objective function: 71.0

0 3 1 2 
6 0 5 1 
4 2 0 3 
5 4 6 0 

Statistic
  status : OPTIMAL
  conflicts: 3238
  branches : 4465
  wall time: 0.8650216850000001 s
  Reasonable cost using OR-Tools: 71.0



([[0, 3, 1, 2], [6, 0, 5, 1], [4, 2, 0, 3], [5, 4, 6, 0]], 71.0)

# Compare the performance of each algorithm.


######`distance_matrix` :

In [ ]:
import random

n = int(input())
distance_matrix = [[random.randint(0,10) for i in range(n)] for i in range(n)]
for i in range(n):
    distance_matrix[i][i] = 0
    for j in range(n):
        distance_matrix[j][i] = distance_matrix[i][j]
print(distance_matrix)

In [ ]:
n

In [ ]:
print(distance_matrix)

### For each algorithm:

In [ ]:
print(beam_solve(distance_matrix,k=10))
print(local_solve(distance_matrix)[1])
print(iterative_local_solve(distance_matrix,K=300)[1])
print(gibb_sampling(distance_matrix,K=1000,factor=10)[1])
print(OR_Tools_solve(distance_matrix, time_limit = 300))

### BeamSearch algorithms isn't fast because of the complexity of `get_sub_node()` func

In [ ]:
print(beam_solve(distance_matrix,k=10))

# Collect data

To collect data from a dataset and test them, connect Colab to Drive, then open *content/drive/MyDrive/Colab Notebooks* and create 2 folders with names: "dataset" and "data".
Upload the datasets into the dataset folder, and then run the code for results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Collect_data functions:

In [ ]:
import time
import csv
import ast

def collectdata_Backtrack(dataset):
    rows = list()
    fieldnames = ['No.', 'Distance_matrix', 'Feasible_solution', 'Cost', 'Solve time (sec)']
    filename = '/content/drive/MyDrive/Colab Notebooks/data/backtrack_' + dataset[:-4] + '.csv'
    num = 0

    with open('/content/drive/MyDrive/Colab Notebooks/dataset/' + dataset, 'r') as data:
        row = data.readline()
        while len(row) > 1:
            row = ast.literal_eval(row)
            start = time.perf_counter()
            Sol, Cost = backtrack_solve(row)
            sol_time = time.perf_counter() - start
            num += 1
            temp = [num, str(row).rstrip() + '\'', str(Sol) + '\'', Cost, sol_time]
            rows.append(temp)
            row = data.readline()

    with open(filename, 'w+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fieldnames)
        writer.writerows(rows)

def collectdata_BeamSearch(dataset, K):
    rows = list()
    fieldnames = ['No.', 'Distance_matrix', 'Feasible_solution', 'Cost', 'Solve time (sec)']
    filename = '/content/drive/MyDrive/Colab Notebooks/data/beam_search_' + dataset[:-4] + '_with_K_' + str(K) + '.csv'
    num = 0

    with open('/content/drive/MyDrive/Colab Notebooks/dataset/' + dataset, 'r') as data:
        row = data.readline()
        while len(row) > 1:
            row = ast.literal_eval(row)
            start = time.perf_counter()
            Sol, Cost = beam_solve(row, K)
            sol_time = time.perf_counter() - start
            num += 1
            temp = [num, str(row).rstrip() + '\'', str(Sol) + '\'', Cost, sol_time]
            rows.append(temp)
            row = data.readline()

    with open(filename, 'w+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fieldnames)
        writer.writerows(rows)

def collectdata_LocalSolve(dataset, i = 1):
    rows = list()
    fieldnames = ['No.', 'Repeat no.', 'Distance_matrix', 'Feasible_solution', 'Cost', 'Solve time (sec)']
    filename = '/content/drive/MyDrive/Colab Notebooks/data/local_solve_' + str(i) + '_' + dataset[:-4] + '.csv'
    num = 0

    with open('/content/drive/MyDrive/Colab Notebooks/dataset/' + dataset, 'r') as data:
        row = data.readline()
        while len(row) > 1:
            row = ast.literal_eval(row)
            num += 1
            for j in range(i):
                start = time.perf_counter()
                Sol, Cost = local_solve(row)
                sol_time = time.perf_counter() - start
                temp = [num, j, str(row).rstrip() + '\'', str(Sol) + '\'', Cost, sol_time]
                rows.append(temp)
            row = data.readline()

    with open(filename, 'w+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fieldnames)
        writer.writerows(rows)

def collectdata_IterLocalSolve(dataset, K, i = 1):
    rows = list()
    fieldnames = ['No.', 'Repeat no.', 'Distance_matrix', 'Feasible_solution', 'Cost', 'Solve time (sec)']
    filename = '/content/drive/MyDrive/Colab Notebooks/data/iterative_local_solve_' + str(i) + '_' + dataset[:-4] + '_with_K_' + str(K) + '.csv'
    num = 0

    with open('/content/drive/MyDrive/Colab Notebooks/dataset/' + dataset, 'r') as data:
        row = data.readline()
        while len(row) > 1:
            row = ast.literal_eval(row)
            num += 1
            for j in range(i):
                start = time.perf_counter()
                Sol, Cost = iterative_local_solve(row, K)
                sol_time = time.perf_counter() - start
                temp = [num, j, str(row).rstrip() + '\'', str(Sol) + '\'', Cost, sol_time]
                rows.append(temp)
            row = data.readline()

    with open(filename, 'w+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fieldnames)
        writer.writerows(rows)

def collectdata_GibbSampling(dataset, K, factor, i = 1):
    rows = list()
    fieldnames = ['No.', 'Repeat no.', 'Distance_matrix', 'Feasible_solution', 'Cost', 'Solve time (sec)']
    filename = '/content/drive/MyDrive/Colab Notebooks/data/gibb_sampling_'+  str(i) + '_' + dataset[:-4] + '_with_K_' + str(K) + '_factor_' + str(factor) + '.csv'
    num = 0

    with open('/content/drive/MyDrive/Colab Notebooks/dataset/' + dataset, 'r') as data:
        row = data.readline()
        while len(row) > 1:
            row = ast.literal_eval(row)

            print(row)
            
            num += 1
            for j in range(i):
                start = time.perf_counter()
                Sol, Cost = gibb_sampling(row, K, factor)
                sol_time = time.perf_counter() - start

                print(sol_time)

                temp = [num, j, str(row).rstrip() + '\'', str(Sol) + '\'', Cost, sol_time]
                rows.append(temp)
            row = data.readline()

    with open(filename, 'w+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fieldnames)
        writer.writerows(rows)

def collectdata_ORTools(dataset, time_limit = 30, i = 1):
    rows = list()
    fieldnames = ['No.','Repeat no.', 'Distance_matrix', 'Feasible_solution', 'Cost', 'Solve time (sec)']
    filename = '/content/drive/MyDrive/Colab Notebooks/data/or_tools_' + str(i) + '_' + dataset[:-4] + '_with_time_limit_' + str(time_limit) + '.csv'
    num = 0

    with open('/content/drive/MyDrive/Colab Notebooks/dataset/' + dataset, 'r') as data:
        row = data.readline()
        while len(row) > 1:
            row = ast.literal_eval(row)
            num += 1
            for j in range(i):
                start = time.perf_counter()
                Sol, Cost = OR_Tools_solve(row, time_limit)
                sol_time = time.perf_counter() - start
                temp = [num, j, str(row).rstrip() + '\'', str(Sol) + '\'', Cost, sol_time]
                rows.append(temp)
            row = data.readline()

    with open(filename, 'w+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fieldnames)
        writer.writerows(rows)



#### Collect the data from each dataset, for each algorithm:

###### Backtrack

In [ ]:
collectdata_Backtrack('dataset_4.txt')

###### BeamSearch

In [ ]:
collectdata_BeamSearch('dataset_4.txt', 10)
collectdata_BeamSearch('dataset_6.txt', 10)
collectdata_BeamSearch('dataset_8.txt', 10)

###### Local Search

In [ ]:
collectdata_LocalSolve('dataset_4.txt', 10)
collectdata_LocalSolve('dataset_6.txt', 10)
collectdata_LocalSolve('dataset_8.txt', 10)
collectdata_LocalSolve('dataset_10.txt', 10)
collectdata_LocalSolve('dataset_12.txt', 10)
collectdata_LocalSolve('dataset_14.txt', 10)
collectdata_LocalSolve('dataset_16.txt', 10)
collectdata_LocalSolve('dataset_18.txt', 10)

###### Iterative Local Search

In [ ]:
collectdata_IterLocalSolve('dataset_4.txt', 20)
collectdata_IterLocalSolve('dataset_6.txt', 20)
collectdata_IterLocalSolve('dataset_8.txt', 20)
collectdata_IterLocalSolve('dataset_10.txt', 20)
collectdata_IterLocalSolve('dataset_12.txt', 20)
collectdata_IterLocalSolve('dataset_14.txt', 20)
collectdata_IterLocalSolve('dataset_16.txt', 20)
collectdata_IterLocalSolve('dataset_18.txt', 20)

###### Gibb Sampling

In [ ]:
collectdata_GibbSampling('dataset_4.txt', 20, 10, 10)
collectdata_GibbSampling('dataset_6.txt', 20, 2, 10)
collectdata_GibbSampling('dataset_8.txt', 20, 2, 10)
collectdata_GibbSampling('dataset_10.txt', 20, 1.5, 10)
collectdata_GibbSampling('dataset_12.txt', 20, 1.25, 10)
collectdata_GibbSampling('dataset_14.txt', 20, 1.125, 10)
collectdata_GibbSampling('dataset_16.txt', 20, 1.0625, 10)
collectdata_GibbSampling('dataset_18.txt', 20, 1.0625, 10)

###### OR-Tools

In [ ]:
collectdata_ORTools('dataset_4.txt', 30)
collectdata_ORTools('dataset_6.txt', 30)
collectdata_ORTools('dataset_8.txt', 60)
collectdata_ORTools('dataset_10.txt', 120)
collectdata_ORTools('dataset_12.txt', 150)
collectdata_ORTools('dataset_14.txt', 150
collectdata_ORTools('dataset_16.txt', 150)
collectdata_ORTools('dataset_18.txt', 150)